This notebook is designed to create a new simulation based on an existing simulation. That we you ensure everything is the same except something in particular.

At first, it is being used to keep all synapses and original background inputs the same while turning clustered inputs into background inputs

Define the simulation you want to base the new one off of

In [1]:
original_sim_dir = '/home/drfrbc/Neural-Modeling/simulations/2025-06-09-12-20-tuft_exc_clusters_30sec_sta/complex'

new_sim_dir = '/home/drfrbc/Neural-Modeling/simulations/2025-06-09-13-55-no_clustering_30sec_sta/complex'

In [2]:
import shutil
import os

import sys
sys.path.append('../')
sys.path.append('../Modules/')


In [3]:
os.chdir('../simulations')

copy the simulation to the new one

In [4]:
# create the new simulation directory if it doesn't exist
if not os.path.exists(new_sim_dir):
    os.makedirs(new_sim_dir)

parameters_file = f'{new_sim_dir}/parameters.pickle'
synapses_file = f'{new_sim_dir}/synapses.csv'

# copy over the parameters and synapses to the new simulation directory
shutil.copyfile(f'{original_sim_dir}/parameters.pickle', parameters_file)
shutil.copyfile(f'{original_sim_dir}/synapses.csv', synapses_file)

'/home/drfrbc/Neural-Modeling/simulations/2025-06-09-13-55-no_clustering_30sec_sta/complex/synapses.csv'

load data

In [5]:
from Modules import analysis
import pandas as pd

# load parameters from the new simulation directory
parameters = analysis.DataReader.load_parameters(new_sim_dir)

# load synapses from the new simulation directory
synapses = pd.read_csv(synapses_file)

--No graphics will be displayed.


### Make your changes to the data

In [6]:
synapses.columns

Index(['name', 'modfile', 'P_0', 'initW', 'cell2cell_type', 'seg_id',
       'gbar_ampa', 'gbar_nmda', 'gbar_gaba', 'spike_train',
       'pc_mean_firing_rate', 'functional_group', 'presynaptic_cell'],
      dtype='object')

In [7]:
# change clustered synapses to background spike trains

# import numpy as np
# from functools import partial
# from Modules.spike_generator import PoissonTrainGenerator
# def assign_background_spike_trains(
#     synapses: pd.DataFrame,
#     parameters,
#     tstop: int,
#     random_state: np.random.RandomState
# ):
#     """
#     Assign background spike trains to provided synapses (assumes all are one type and sec_type).
#     synapses: DataFrame with at least columns 'name', 'spike_train', 'pc_mean_firing_rate'
#     parameters: parameters object with .exc_syn_properties and .inh_syn_properties
#     tstop: simulation stop time (int)
#     random_state: numpy random state for reproducibility
#     """
#     # This assumes that each synapse's type can be determined from the 'name' column
#     new_synapses = synapses.copy()
#     for idx, row in new_synapses.iterrows():
#         # Determine synapse type and sec_type (refactor this as needed)
#         if 'exc' in row['name']:
#             syn_type = 'exc'
#             prop_dict = parameters.exc_syn_properties
#         elif 'inh' in row['name']:
#             syn_type = 'inh'
#             prop_dict = parameters.inh_syn_properties
#         else:
#             raise ValueError(f"Unknown synapse type in name: {row['name']}")

#         # Find sec_type: this could be parsed from 'name' or added as a column earlier
#         # This assumes sec_type is part of the name, e.g. 'exc_apical'
#         sec_type = None
#         for key in prop_dict.keys():
#             if key in row['name']:
#                 sec_type = key
#                 break
#         if sec_type is None:
#             raise ValueError(f"Cannot determine sec_type for {row['name']}")

#         # Get the mean firing rate distribution for this type/sec_type
#         mean_fr_dist = prop_dict[sec_type]['mean_firing_rate_distribution']
#         mean_fr_sampler = partial(mean_fr_dist['function'], **mean_fr_dist['params'], size=1)

#         # Generate a single timecourse (flat or pink noise, here we use flat)
#         background_firing_rate_timecourse = np.ones(tstop)
#         pc_mean_firing_rate = mean_fr_sampler(size=1)

#         # Generate timecourse shifted to correct mean
#         pc_firing_rate_timecourse = PoissonTrainGenerator.shift_mean_of_lambdas(
#             lambdas=background_firing_rate_timecourse,
#             desired_mean=pc_mean_firing_rate
#         )

#         # Generate Poisson spike train
#         spike_train = PoissonTrainGenerator.generate_spike_train(
#             lambdas=pc_firing_rate_timecourse,
#             random_state=random_state
#         )
#         new_synapses.at[idx, 'spike_train'] = np.array(spike_train.spike_times)
#         new_synapses.at[idx, 'pc_mean_firing_rate'] = pc_mean_firing_rate

#     return new_synapses

import numpy as np
import pandas as pd
from functools import partial
from Modules.spike_generator import PoissonTrainGenerator
import concurrent.futures

def process_row(args):
    idx, row, parameters, tstop, random_state = args
    # Synapse type
    if 'exc' in row['name']:
        prop_dict = parameters.exc_syn_properties
    elif 'inh' in row['name']:
        prop_dict = parameters.inh_syn_properties
    else:
        raise ValueError(f"Unknown synapse type in name: {row['name']}")
    sec_type = next((key for key in prop_dict.keys() if key in row['name']), None)
    if sec_type is None:
        raise ValueError(f"Cannot determine sec_type for {row['name']}")
    mean_fr_dist = prop_dict[sec_type]['mean_firing_rate_distribution']
    mean_fr_sampler = partial(mean_fr_dist['function'], **mean_fr_dist['params'], size=1)
    background_firing_rate_timecourse = np.ones(tstop)
    pc_mean_firing_rate = mean_fr_sampler(size=1)
    pc_firing_rate_timecourse = PoissonTrainGenerator.shift_mean_of_lambdas(
        lambdas=background_firing_rate_timecourse,
        desired_mean=pc_mean_firing_rate
    )
    spike_train = PoissonTrainGenerator.generate_spike_train(
        lambdas=pc_firing_rate_timecourse,
        random_state=random_state
    )
    return idx, np.array(spike_train.spike_times), pc_mean_firing_rate

def assign_background_spike_trains_parallel(
    synapses: pd.DataFrame,
    parameters,
    tstop: int,
    random_state: np.random.RandomState
):
    new_synapses = synapses.copy()
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(
            process_row,
            [(idx, row, parameters, tstop, random_state) for idx, row in new_synapses.iterrows()]
        ))
    for idx, spike_train, pc_mean_firing_rate in results:
        new_synapses.at[idx, 'spike_train'] = spike_train
        new_synapses.at[idx, 'pc_mean_firing_rate'] = pc_mean_firing_rate
    return new_synapses


import numpy as np

# Save a copy of your original DataFrame
synapses_original = synapses.copy()


# Assuming synapses is loaded and parameters is available
random_state = np.random.RandomState(42)  # or whatever seed you want

# Filter for clustered synapses
clustered_mask = ((synapses['functional_group'] != -1.0) | (synapses['presynaptic_cell'] != -1.0))
clustered_synapses = synapses[clustered_mask].copy()

# # Check example spike trains before
# print("Old spike trains for clustered synapses:")
# print(synapses.loc[clustered_synapses.index, 'spike_train'].head())

# Reassign background spike trains
tstop = parameters.h_tstop
backgrounded = assign_background_spike_trains_parallel(clustered_synapses, parameters, tstop, random_state)

# Save or overwrite in the original DataFrame if needed:
synapses.loc[clustered_synapses.index, ['spike_train', 'pc_mean_firing_rate']] = \
    backgrounded[['spike_train', 'pc_mean_firing_rate']]

# Save back to disk if needed
synapses.to_csv(os.path.join(new_sim_dir, "synapses.csv"), index=False)

# verify the changes
# Check after
# Print BEFORE and AFTER for the SAME indices!
for idx in clustered_synapses.index[:5]:
    print(f"Index {idx}:")
    print("  Original:", synapses_original.at[idx, 'spike_train'])
    print("  Current:", synapses.at[idx, 'spike_train'])



Index 0:
 Original:
  Old: [  124   233   968  1881  1915  2597  3091  4676  4917  5371  5848  6652
  6883  8060  8196 11891 12137 13739 14834 15552 16029 16744 17301 18269
 19221 19285 20686 21057 21838 23954 24098 24335 24362 24521 24880 25202
 26105 26309 26749 27669 28341 28914 29903]
  New: [  531  3648 10947 14232 15021 19599 20214 20500 20876 21042 24823 28957
 29155]
Index 1:
 Original:
  Old: [   41   501   575   691   902   925  1397  1440  1498  1703  1822  1875
  2033  2105  2272  2377  2406  2521  2583  2596  2652  2763  2772  2980
  3013  3110  3175  3349  3483  3496  3509  3840  3887  3895  4216  4408
  4617  4681  4822  4988  5297  5467  5537  5567  5627  5875  6031  6055
  6110  6477  6534  6555  6718  7188  7282  7286  7541  7616  7623  7763
  7917  8150  8207  8530  8590  8636  8668  8723  8795  9528  9653  9801
 10085 10434 10525 10907 10946 11073 11442 12021 12671 12717 12806 12855
 13593 13682 13847 14186 14229 14280 14690 14730 14786 15079 15080 15526
 15554 1557

In [8]:
len(np.unique(synapses['spike_train']))

TypeError: '<' not supported between instances of 'str' and 'numpy.ndarray'